必需要的資料:

*熱門遊戲種類
*1."種類名稱" 2."遊戲名稱" 3."所屬公司" 4."發行日期" 5."評分" 6."遊戲介紹"

*遊戲新聞
*1."標題名稱" 2."時間" 3."地點" 4."新聞內容"

*展覽資訊
*1."主題名稱" 2."時間" 3."地點" 4."展覽內容"

In [78]:
def open_db():#開啟Database
    import pymysql.cursors
    db = pymysql.connect(host='deepbowl-mysql.cukfiecfgfog.ap-northeast-1.rds.amazonaws.com',#連結MySQL
                             user='public_root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return db

In [79]:
def search_db(table=None, columns = None, limit = None,operation = None, keyword = None, quantity=None):
    #talbe:資料表可能會有Join在一起的表，不過這是用字串儲存
    #columns:欄位的名稱
    #limit:篩選條件
    #keyword:要查的關鍵字,Ex:遊戲名等
    #quantity:目前無
    db = open_db()
    try:
        cursor = db.cursor()#建立資料庫游標
        sql = ("select "+columns+" \
                from "+table+" \
                where "+limit+" "+operation+" '"+keyword+"' ")#下指令
        
        cursor.execute(sql)
        rows = cursor.fetchall()
        return rows
    except Exception as e:
        print("Exeception occured:{}".format(e))
    finally:
        db.close()#關閉DataBase

In [89]:
def game_type(game_type = None):#查詢遊戲類型
    import random
    table = 'type inner join (game inner join game_has_type using (game_ID)) using (type_ID)'#目標Table
    columns = 'distinct(game_name)'#詢問遊戲類型時有的columns:遊戲名稱
    game_type = '%'.join(i for i in game_type)#字與字之間插入，SQL like語法要用的'%'符號
    game_type = '%' + game_type + '%'#頭跟尾加上SQL like語法要用的'%'符號
    ans = search_db(table,columns, limit = 'type_name',operation = 'like', keyword = game_type)#進資料庫搜尋的各項參數設定
    random.shuffle(ans)#打亂查詢到遊戲名稱的list
    if ans:
        ans = '  ,'.join(i['game_name'] for i in ans[0:5])#取前5項的遊戲名稱並用'，'合併
    else:
        ans = 'haha none'
    return ans

In [80]:
def game_introduction(game_name = None):#遊戲介紹的Funtion
    table = 'game'#目標Table
    columns = 'game_description,game_release_date'#介紹遊戲時有的columns:介紹內容、發行日期
    game_name = '%'.join(i for i in game_name)#字與字之間插入，SQL like語法要用的'%'符號
    game_name = '%' + game_name + '%'#頭跟尾加上SQL like語法要用的'%'符號
    ans = search_db(table,columns, limit = 'game_name', operation = 'like',keyword = game_name)#進資料庫搜尋的各項參數設定
    if ans:
        ans = str(ans[0]['game_description']) + 'Release Date: ' + ans[0]['game_release_date'].strftime('%Y/%m/%d')#合併columns，DateTime類型轉String使用strftime
    else:
        ans = 'haha none',
    return ans

In [90]:
import json
from flask import Flask, request, make_response, jsonify
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pymysql.cursors

app = Flask(__name__)


@app.route("/")
def verify():
    return "Hello world", 200


@app.route('/', methods=['POST'])#Dialogflow端
def webhook():
    req = request.get_json(silent=True, force=True)
    print(req)
    retJson = ''#查詢完的結果Asign到這兒

    
    if req['queryResult']['intent']['displayName'] != None:#確認Intent是否存在
        targetIntent = req['queryResult']['intent']['displayName']#儲存Intent的名稱  
        
    if targetIntent == 'askGameIntroduction':#Intent的名稱是遊戲介紹
        if req['queryResult']['parameters']['game_introduction'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_introduction(game_name = str(keyword))
            
    if targetIntent == 'askGameType':#Intent的名稱是遊戲類型
        if req['queryResult']['parameters']['typeflag'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_type(game_type = str(keyword))
    res = {"fulfillmentText": retJson}#回傳DialogFlow的Json
    res = json.dumps(res,indent = 4)
    print(res)
    return res

# @app.route('/lastgame', methods=['GET'])#web端
# def webView():
#     cursor = db.cursor()#建立資料庫游標
    
#     sql = ("select * from game")#下指令
    
#     cursor.execute(sql)
#     rows = cursor.fetchall()
    
    
#     req = request.get_json(silent=True, force=True)
#     print(req)
    
#     return res

if __name__ == '__main__':
    app.run(port=5000)
    

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'responseId': '9aaf191c-ee0f-4e75-9de4-f56660c38695', 'queryResult': {'queryText': '給我射擊的遊戲類型', 'parameters': {'any': '射擊', 'typeflag': '遊戲類型'}, 'allRequiredParamsPresent': True, 'fulfillmentText': '沒有此類型遊戲', 'fulfillmentMessages': [{'text': {'text': ['沒有此類型遊戲']}, 'platform': 'LINE'}, {'text': {'text': ['沒有此類型遊戲']}}], 'intent': {'name': 'projects/newagent-bc844/agent/intents/283c7eb5-27c4-4c85-a9cc-e0555e7064ae', 'displayName': 'askGameType'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/newagent-bc844/agent/sessions/2ff2ac85-aee9-c165-adc7-5de790691dd0'}
askGameType
2
射擊


127.0.0.1 - - [19/Dec/2018 11:31:32] "POST / HTTP/1.1" 200 -


[{'game_name': 'Dying Light'}, {'game_name': 'Hero Siege'}, {'game_name': 'Subnautica'}, {'game_name': 'Tomb Raider'}, {'game_name': 'METAL GEAR SOLID V: GROUND ZEROES'}]
{
    "fulfillmentText": "Dying Light  ,Hero Siege  ,Subnautica  ,Tomb Raider  ,METAL GEAR SOLID V: GROUND ZEROES"
}
{'responseId': '46a4fddb-50cc-43b4-bb5f-7caf1b4773e5', 'queryResult': {'queryText': '模擬類型', 'parameters': {'any': '模擬', 'typeflag': '類型'}, 'allRequiredParamsPresent': True, 'fulfillmentText': '沒有此類型遊戲', 'fulfillmentMessages': [{'text': {'text': ['沒有此類型遊戲']}, 'platform': 'LINE'}, {'text': {'text': ['沒有此類型遊戲']}}], 'intent': {'name': 'projects/newagent-bc844/agent/intents/283c7eb5-27c4-4c85-a9cc-e0555e7064ae', 'displayName': 'askGameType'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/newagent-bc844/agent/sessions/2ff2ac85-aee9-c165-adc7-5de790691dd0'}
askGameType
2
模擬


127.0.0.1 - - [19/Dec/2018 11:31:56] "POST / HTTP/1.1" 200 -


[{'game_name': 'SENRAN KAGURA ESTIVAL VERSUS'}, {'game_name': 'Graveyard Keeper'}, {'game_name': 'Trailmakers'}, {'game_name': 'Assassin’s Creed® Rogue'}, {'game_name': 'Planet Coaster'}]
{
    "fulfillmentText": "SENRAN KAGURA ESTIVAL VERSUS  ,Graveyard Keeper  ,Trailmakers  ,Assassin\u2019s Creed\u00ae Rogue  ,Planet Coaster"
}
{'responseId': 'aa963c3a-99b9-4425-9af2-8d1f9f7815de', 'queryResult': {'queryText': 'UNO的介紹', 'parameters': {'game_introduction': '介紹', 'any': 'UNO'}, 'allRequiredParamsPresent': True, 'fulfillmentText': '試試看搜尋別的遊戲吧', 'fulfillmentMessages': [{'text': {'text': ['沒有此遊戲的介紹']}, 'platform': 'LINE'}, {'text': {'text': ['試試看搜尋別的遊戲吧']}}], 'intent': {'name': 'projects/newagent-bc844/agent/intents/090765e8-1142-4762-a22f-c5a9c053b954', 'displayName': 'askGameIntroduction'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/newagent-bc844/agent/sessions/2ff2ac85-aee9-c165-adc7-5de790691dd0'}
a

127.0.0.1 - - [19/Dec/2018 11:32:47] "POST / HTTP/1.1" 200 -


{
    "fulfillmentText": "\u7d55\u5730\u6c42\u751f(PLAYERUNKNOWN\u2019S BATTLEGROUNDS)\u662f\u5927\u9003\u6bba\u985e\u578b\u7684\u904a\u6232\uff0c\u6bcf\u4e00\u5c40\u904a\u6232\u5c07\u6709100\u540d\u73a9\u5bb6\u53c3\u8207\uff0c \u4ed6\u5011\u5c07\u88ab\u6295\u653e\u5728\u7d55\u5730\u5cf6(battlegrounds)\u7684\u4e0a\u7a7a\uff0c\u904a\u6232\u958b\u59cb\u8df3\u5098\u6642\u6240\u6709\u4eba\u90fd\u4e00\u7121\u6240\u6709\u3002Release Date: 2017/12/21"
}
